In [1]:
import numpy as np
from PIL import Image
from pathlib import Path

In [2]:
# Set up parameters
grid_size = (23, 23)  # Adjust grid size as needed
resize_factor = 0.4   # Resize factor to reduce image dimensions

# Define input and output paths
input_folder = Path("recon")             # Path to the recon/ folder
output_image_path = Path("output_grid.png")  # Save output in the main folder

# Check if the input folder exists
if not input_folder.exists():
    print(f"Input folder '{input_folder}' does not exist.")
else:
    print(f"Input folder '{input_folder}' found.")

Input folder 'recon' found.


In [3]:
# Function to get required images
def get_required_images(folder_path, grid_size):
    required_images = grid_size[0] * grid_size[1]
    valid_images = []

    for file_path in folder_path.glob("*.tif"):
        valid_images.append(file_path)
        if len(valid_images) >= required_images:
            break
    return valid_images[:required_images]

In [4]:
# Function to load and normalize images
def load_and_normalize_images(image_paths, resize_factor=1.0):
    images = []
    global_min = float('inf')
    global_max = float('-inf')

    # First pass: Determine global min and max
    for file_path in image_paths:
        img = Image.open(file_path)
        if img.mode != 'I;16':
            img = img.convert('I;16')

        img_array = np.array(img)
        min_val = np.min(img_array)
        max_val = np.max(img_array)

        if min_val < global_min:
            global_min = min_val
        if max_val > global_max:
            global_max = max_val

    # Second pass: Load and normalize images
    for file_path in image_paths:
        img = Image.open(file_path)
        img = img.convert('I;16')
        img_array = np.array(img)

        # Normalize to [0, 1] range based on global min and max
        img_normalized = (img_array - global_min) / (global_max - global_min)
        img_8bit = (img_normalized * 255).astype(np.uint8)

        img = Image.fromarray(img_8bit)

        if resize_factor != 1.0:
            new_size = (int(img.width * resize_factor), int(img.height * resize_factor))
            img = img.resize(new_size, Image.BICUBIC)
            img_8bit = np.array(img)

        images.append(img_8bit)

    if not images:
        return np.array([])

    return np.stack(images, axis=0)

In [5]:
# Function to save slices as a grid image
def save_slices_as_grid_image(normalized_data, output_path, grid_size):
    if normalized_data.size == 0:
        print("No valid images to create a grid.")
        return

    num_slices = normalized_data.shape[0]
    slice_height, slice_width = normalized_data.shape[1:3]

    # Create an empty image for the grid
    grid_image = Image.new('L', (slice_width * grid_size[1], slice_height * grid_size[0]))

    for i in range(num_slices):
        row = i // grid_size[1]
        col = i % grid_size[1]

        if row >= grid_size[0]:
            break

        slice_data = normalized_data[i, :, :]
        slice_image = Image.fromarray(slice_data)
        grid_image.paste(slice_image, (col * slice_width, row * slice_height))

    grid_image.save(output_path)
    print(f"Grid image saved to {output_path}")

In [6]:
# Get the required image paths
if input_folder.exists():
    image_paths = get_required_images(input_folder, grid_size)
    if not image_paths:
        print("No valid images found to create a grid.")
    else:
        print(f"Found {len(image_paths)} images.")

Found 529 images.


In [7]:
# Load and normalize images
normalized_data = load_and_normalize_images(image_paths, resize_factor=resize_factor)

In [8]:
# Save the grid image
save_slices_as_grid_image(normalized_data, output_image_path, grid_size)

Grid image saved to output_grid.png


In [ ]:
image_paths # TODO sort the paths!

[PosixPath('recon/Insects_I_000870.tif'),
 PosixPath('recon/Insects_I_000864.tif'),
 PosixPath('recon/Insects_I_000694.tif'),
 PosixPath('recon/Insects_I_000858.tif'),
 PosixPath('recon/Insects_I_000680.tif'),
 PosixPath('recon/Insects_I_000657.tif'),
 PosixPath('recon/Insects_I_000643.tif'),
 PosixPath('recon/Insects_I_000496.tif'),
 PosixPath('recon/Insects_I_000482.tif'),
 PosixPath('recon/Insects_I_001011.tif'),
 PosixPath('recon/Insects_I_000469.tif'),
 PosixPath('recon/Insects_I_001005.tif'),
 PosixPath('recon/Insects_I_000333.tif'),
 PosixPath('recon/Insects_I_000455.tif'),
 PosixPath('recon/Insects_I_000441.tif'),
 PosixPath('recon/Insects_I_000327.tif'),
 PosixPath('recon/Insects_I_001039.tif'),
 PosixPath('recon/Insects_I_000284.tif'),
 PosixPath('recon/Insects_I_000290.tif'),
 PosixPath('recon/Insects_I_000509.tif'),
 PosixPath('recon/Insects_I_000247.tif'),
 PosixPath('recon/Insects_I_000521.tif'),
 PosixPath('recon/Insects_I_000535.tif'),
 PosixPath('recon/Insects_I_000253